In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from skimage.morphology import disk
from skimage import morphology
from skimage.filters import threshold_otsu, rank
from skimage import io
from os.path import isfile, join
from os import listdir
from skimage.feature import blob_dog, blob_log, blob_doh
from skimage.draw import circle
import numpy as np

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)
#pylab.rcParams['figure.figsize'] = 10,10 
runBlobdetection = False

In [ ]:
dirName = '/home/psnegi/psn/UH/Research/Projects/EdgeOrientation/Data/AngleJData'
#dirName = '/home/psnegi/psn/UH/Research/Projects/EdgeOrientation/Data/DentateGyrus'
images = [ io.imread(join(dirName,f)) for f in listdir(dirName) if
               isfile(join(dirName,f))]
names = [ f for f in listdir(dirName) if
               isfile(join(dirName,f))]

radius = 20
smallObjectSz = 80
blobSigma= 2*smallObjectSz
selem = disk(radius)
localOstuImages = []
globalOstuImages = []
fontSz = 30
for img in images:
    local_otsu = rank.otsu(img, selem)
    threshold_global_otsu = threshold_otsu(img)
    
    fig, ax = plt.subplots(3, 2, figsize=(45, 45), sharex=True, sharey=True, subplot_kw={'adjustable':'box-forced'})
    ax1, ax2, ax3, ax4, ax5, ax6 = ax.ravel()

    fig.colorbar(ax1.imshow(img, cmap=plt.cm.gray),
             ax=ax1, orientation='horizontal')
    ax1.set_title('Original', fontsize=fontSz)
    ax1.axis('off')

    fig.colorbar(ax2.imshow(local_otsu, cmap=plt.cm.gray),
             ax=ax2, orientation='horizontal')
    ax2.set_title('Local Otsu (radius={}) threshold map'.format(radius), fontsize=fontSz)
    ax2.axis('off')
    
    localOstu = img >= local_otsu
    ax3.imshow(localOstu, cmap=plt.cm.gray)
    ax3.set_title('Original >= Local Otsu', fontsize=fontSz)
    ax3.axis('off')
    
    localOstu =morphology.remove_small_objects(localOstu, min_size = smallObjectSz, connectivity=2)
    
    ax4.imshow(localOstu, cmap=plt.cm.gray)
    ax4.set_title('Original >= Local Otsu after removing small object of size {}'.format(smallObjectSz), fontsize=fontSz)
    ax4.axis('off')
    localOstuImages.append(localOstu)
    
    global_otsu = img >= threshold_global_otsu
    
    ax5.imshow(global_otsu, cmap=plt.cm.gray)
    ax5.set_title('Global Otsu (threshold = {})'.format(threshold_global_otsu), fontsize=fontSz)
    ax5.axis('off')
    
    global_otsu=morphology.remove_small_objects(global_otsu, min_size = smallObjectSz, connectivity=2)
    ax6.imshow(global_otsu, cmap=plt.cm.gray)
    ax6.set_title('Global Otsu (threshold = {}) after removing small object of size {}'.format(threshold_global_otsu,smallObjectSz), fontsize=fontSz)
    ax6.axis('off')
    globalOstuImages.append(global_otsu)
    
plt.show()

    

# blob removal (make condition True if  required for blob removal)
**Note range of sigma play a critical role in blob detection**

In [ ]:
def removeBlobs(imagesWithBlob):    
    minSigma = [20, 20,60]
    maxSigma = [40, 40, 90]
    imageSeq = zip(minSigma, maxSigma, imagesWithBlob)
    cleanImages = []
    for minSig, maxSig, image in imageSeq:
        blobs_doh = blob_log(image, min_sigma=minSig , max_sigma=maxSig, num_sigma=3, threshold=.1)
    # Compute radii in the 3rd column.
        fig, ax = plt.subplots(1, 1)
        #ax.set_title(title)
        ax.imshow(image, interpolation='nearest')
        for blob in blobs_doh:
            y, x, r = blob
            c = plt.Circle((x, y), r, color='yellow', linewidth=2, fill=False)
            ax.add_patch(c)
            rr, cc = circle(y, x, r)
            for x , y in zip(rr, cc):
                if x< image.shape[0] and y < image.shape[1]:
                    image[x, y] =0     
            #rr = np.array([ i for i in rr if i < image.shape[0]], dtype= np.int64)
            #cc = np.array([ i for i in cc if i < image.shape[1]], dtype= np.int64)
            #image[rr, cc] = 0
        cleanImages.append(image)
        plt.show()
    return cleanImages            
if runBlobdetection:
    cleanImagesLocalOstu = removeBlobs(localOstuImages)# decide use localOstuImages or globalOstuImages
    cleanImagesGlobalOstu = removeBlobs(globalOstuImages)# decide use localOstuImages or globalOstuImages
else:
    cleanImagesLocalOstu = localOstuImages
    cleanImagesGlobalOstu = globalOstuImages
    


# save the data

In [ ]:
for ori , segL, segG, f  in zip( images, localOstuImages,globalOstuImages , names):
        
    plt.imshow(ori, cmap=plt.cm.gray)
    plt.title(f)
    plt.show()
    plt.imshow(segL, cmap=plt.cm.gray)
    plt.title('Local ostu')
    plt.show()
    plt.imshow(segG, cmap=plt.cm.gray)
    plt.title('Global ostu')    
    plt.show()
    io.imsave(join(dirName,  f[:f.find('.')]+'_LOstuSeg' + '.png'), segL)
    io.imsave(join(dirName,  f[:f.find('.')]+'_GOstuSeg' + '.png'), segG)